In [ ]:
import os
import sys
import numpy as np
from pickle import dumps
from fonty.font import Font
from bson.binary import Binary
from pymongo import MongoClient

sys.path.insert(1, os.path.join(sys.path[0], '..', 'fonty'))

SIZE = 64

In [ ]:
cli = MongoClient(
    'mongodb+srv://fonty.hquocfa.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority',
    tlsCertificateKeyFile='auth/X509-cert-6658948524510096383.pem',
    tls=True,
)

glyph_col = cli.fonty.glyphs
font_col = cli.fonty.fonts

In [ ]:
def Payload(font: Font, lat: np.ndarray, cyr: np.ndarray):
    """Returns a database payload with arrays stored in binary form."""

    return {
        'name': font.font_name,
        'panose': font.panose,
        'lat': Binary(dumps(lat, protocol=2)),
        'cyr': Binary(dumps(cyr, protocol=2)),
        'size': SIZE,
    }


In [ ]:

def getIds(col):
    for x in col.find({}, {'_id': True}):
        yield x['_id']


def getFonts(col, ids):
    for id in ids:
        yield Font(col, {'_id': id}, SIZE, SIZE)


Do all alphabet glyphs have the same indexing in different fonts?

- Latin Characters: 32-57, 62-87
- Cyrillic Characters: 90-155

Code for manual inspection:
```python
import pylab as plt

for i in range(0, 155):
    print(i)
    plt.imshow(font.glyphs[i].np_array)
    plt.show()
```

In [ ]:
def getPayloads(fonts):
    for font in fonts:
        try:
            lat = []
            lat.extend(font.glyphs[i].np_array for i in range(32, 58))
            lat.extend(font.glyphs[i].np_array for i in range(62, 88))
            lat = np.array(lat)[:,:,:,0].reshape(-1)
        except:
            lat = np.empty(0)

        try:
            cyr = []
            cyr.extend(font.glyphs[i].np_array for i in range(90, 156))
            cyr = np.array(cyr)[:,:,:,0].reshape(-1)
        except:
            cyr = np.empty(0)

        yield Payload(font, lat, cyr)


In [ ]:
ids = getIds(font_col)
fonts = getFonts(font_col, ids)
payloads = getPayloads(fonts)

In [ ]:
glyph_col.delete_many({'size': SIZE})
glyph_col.insert_many(payloads)